### FIRST 

In [ ]:
import sys
grammer = {}

non_terminals = set()

first_sets = {}

In [ ]:
def parse_grammer(input_lines):
  grammer.clear()
  non_terminals.clear()

  for line in input_lines:
    line = line.strip()
    if not line or line.startswith('#'):
      continue
    head, _, str_productions = line.partition(':')
    head = head.strip()

    if not head.isupper():
      print(f" Non terminal {head} is not uppercase.")

    non_terminals.add(head)

    if head not in grammer:
      grammer[head] = []

    productions = str_productions.split('|')
    for production in productions:
      production = production.strip()
      grammer[head].append(production.split()) 

In [ ]:
def calculate_first_for_sequence(sequence):
  first = set()

  if not sequence or (len(sequence) == 1 and sequence[0] == 'Epsilon'):
    return {'Epsilon'}

  can_produce_epsilon = True

  for symbol in sequence:
    if symbol not in non_terminals:
      first.add(symbol)
      can_produce_epsilon = False
      break

    first_of_symbol = first_sets.get(symbol, set())
    first.update(first_of_symbol - {'Epsilon'})

    if 'Epsilon' not in first_of_symbol:
      can_produce_epsilon = False
      break

  if can_produce_epsilon:
    first.add('Epsilon')

  return first


In [ ]:
def compute_first_set():

  for nt in non_terminals:
    first_sets[nt] = set()

  while True:
    changed = False

    for nt in non_terminals:
      original_size = len(first_sets[nt])

      for production in grammer.get(nt, []):
        first_of_rhs = calculate_first_for_sequence(production)
        first_sets[nt].update(first_of_rhs)

      if len(first_sets[nt]) != original_size:
        changed = True

    if not changed:
      break


In [ ]:
def main():

  print("Enter the Grammer Rules: ")
  print("use 'Epsilon'. press Enter on an Empty line when you are done")
  print("-" * 20)

  input_lines = []
  while True:
    try:
      line = input()
      if not line:
        break
      input_lines.append(line)

    except EOFError:
      break

  if not input_lines:
    print("No grammer rules for Entered. Exiting")
    return

  try:

    parse_grammer(input_lines)

    compute_first_set()

    print("\nFIRST Sets:")
    print("-" * 20)
    sorted_nts = sorted(list(non_terminals)) 
    for nt in sorted_nts:

            terminals_str = ", ".join(sorted(list(first_sets.get(nt, set()))))
            print(f"FIRST({nt}) = {{ {terminals_str} }}")

  except Exception as e:
        print(f"\nAn error occurred: {e}", file=sys.stderr)

In [ ]:
if __name__ == "__main__":
    main()

Enter the Grammer Rules: 
use 'Epsilon'. press Enter on an Empty line when you are done
--------------------
S : A | B C
A : a | b
B : p | Epsilon
C : c


FIRST Sets:
--------------------
FIRST(A) = { a, b }
FIRST(B) = { Epsilon, p }
FIRST(C) = { c }
FIRST(S) = { a, b, c, p }


### Follow

In [ ]:
def compute_follow_set():

  follow_sets = {nt: set() for nt in non_terminals}

  start_symbol = list(grammer.keys())[0]
  follow_sets[start_symbol].add('$')


  changed = True
  while changed:
    changed = False


    for head, productions in grammer.items():
      for production in productions:
        for i, symbol in enumerate(production):
          if symbol in non_terminals:
            beta = production[i+1:]
            first_of_beta = calculate_first_for_sequence(beta)

            original_size = len(follow_sets[symbol])
            follow_sets[symbol].update(first_of_beta - {'Epsilon'})

            if 'Epsilon' in first_of_beta or not beta:
              follow_sets[symbol].update(follow_sets[head])

            if len(follow_sets[symbol]) != original_size:
              changed = True 
  return follow_sets 

#### Construct LL(1) parsing table


In [ ]:
def construct_ll1_table(grammer, first_sets, follow_sets):
  parsing_table = {}

  terminals = set()
  for productions in grammer.values():
    for production in productions:
      for symbol in production:
        if symbol not in non_terminals and symbol != 'Epsilon':
          terminals.add(symbol)
  terminals.add('$')

  for nt in non_terminals:
    parsing_table[nt] = {}
    for t in terminals:
      parsing_table[nt][t] = None

  for head, productions in grammer.items():
    for production in productions:
      first_of_rhs = calculate_first_for_sequence(production)

      if 'Epsilon' in first_of_rhs:
        for terminal in follow_sets.get(head, set()):
           if parsing_table[head][terminal] is not None:
             print(f"Conflict at M[{head}, {terminal}]: attempting to add {production} but {parsing_table[head][terminal]} already exists.")
           parsing_table[head][terminal] = production
      else:
        for terminal in first_of_rhs:
          if terminal != 'Epsilon':
            if parsing_table[head][terminal] is not None:
              print(f"Conflict at M[{head}, {terminal}]: attempting to add {production} but {parsing_table[head][terminal]} already exists.")
            parsing_table[head][terminal] = production

  return parsing_table

#### Display the computed FOLLOW sets and the constructed LL(1) parsing table.


In [ ]:

follow_sets = compute_follow_set()
parsing_table = construct_ll1_table(grammer, first_sets, follow_sets)

print("\nFOLLOW Sets:")
print("-" * 20)
sorted_nts = sorted(list(non_terminals)) 
for nt in sorted_nts:
    terminals_str = ", ".join(sorted(list(follow_sets.get(nt, set()))))
    print(f"FOLLOW({nt}) = {{ {terminals_str} }}")

print("\nLL(1) Parsing Table:")
print("-" * 20)

terminals = set()
for productions in grammer.values():
    for production in productions:
        for symbol in production:
            if symbol not in non_terminals and symbol != 'Epsilon':
                terminals.add(symbol)
terminals.add('$')
sorted_terminals = sorted(list(terminals))

header = "{:<10}".format("Non-Terminal")
for t in sorted_terminals:
    header += "{:<10}".format(t)
print(header)
print("-" * (10 * (len(sorted_terminals) + 1)))


for nt in sorted_nts:
    row = "{:<10}".format(nt)
    for t in sorted_terminals:
        production = parsing_table.get(nt, {}).get(t)
        if production is not None:
            production_str = " ".join(production)
            row += "{:<10}".format(f"{nt} -> {production_str}")
        else:
            row += "{:<10}".format("")
    print(row)


FOLLOW Sets:
--------------------
FOLLOW(A) = { $ }
FOLLOW(B) = { c }
FOLLOW(C) = { $ }
FOLLOW(S) = { $ }

LL(1) Parsing Table:
--------------------
Non-Terminal$         a         b         c         p         
------------------------------------------------------------
A                   A -> a    A -> b                        
B                                       B -> EpsilonB -> p    
C                                       C -> c              
S                   S -> A    S -> A    S -> B C  S -> B C  
